In [1]:
# Compare three different algorithms: Dynamic Pricing, Adaptive Pricing, and FCFS
# Use intercepts and slopes from initialization.py as starting point for linear demand curve
# Dynamic Pricing: 
    # Retail Price Optimization at InterContinental Hotels Group. 
    # INFORMS Journal on Applied Analytics 42(1):45-57. 
    # https://doi.org/10.1287/inte.1110.0620

# Adaptibe Pricing: Developed by me, adapted from:
    # Revenue Management Without Forecasting or Optimization: An Adaptive Algorithm for Determining Airline Seat Protection Levels
    # Management Science 46(6):760-775.
    # https://doi.org/10.1287/mnsc.46.6.760.11936
    
# FCFS: First-Come, First-Serve

In [2]:
import itertools
import numpy as np
from scipy.optimize import linprog
from cvxopt import matrix, solvers, spmatrix
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

from initialize import linparams

In [3]:
# Parameter values
n_class = 2
los = 3
capacity = 50
intensity = 1.5
slope_init = np.array([-0.1, -0.15])
rates_init = np.array([[135, 135, 135, 135, 135, 108, 108],
                       [115, 115, 115, 115, 115, 92, 92]])
combs = n_class * 7 * los

In [4]:
# intercepts and slopes for rate class, arrival day of week and los combination
params = linparams(capacity, intensity, slope_init, rates_init)

In [5]:
# Starting parameters
params_slopes = params[0]
params_intercepts = params[1]

In [6]:
params_slopes

array([[[-0.05214334, -0.10577779, -0.08281085],
        [-0.10717032, -0.07648435, -0.07894849],
        [-0.0840614 , -0.0794162 , -0.06407827],
        [-0.11011302, -0.09129206, -0.07715908],
        [-0.06242003, -0.07844733, -0.08082723],
        [-0.12932441, -0.09954261, -0.09987862],
        [-0.07289733, -0.09943966, -0.09310046]],

       [[-0.20080256, -0.1323451 , -0.10682807],
        [-0.12603916, -0.10804363, -0.09130725],
        [-0.14316521, -0.09202643, -0.09830205],
        [-0.16671679, -0.1076695 , -0.08507253],
        [-0.11504746, -0.10273544, -0.10389833],
        [-0.16421323, -0.11944227, -0.09195448],
        [-0.12111864, -0.09594896, -0.09979728]]])

In [7]:
# Calculate averate rates for each arrival day of week and los combination
rates_arrival_los = [[rates_init[i, j],
                      rates_init[i, j] + rates_init[i, (j+1)%7],
                      rates_init[i, j] + rates_init[i, (j+1)%7] + rates_init[i, (j+2)%7]] 
                      for i, j in itertools.product(range(n_class), range(7))]
# Store it as a numpy array
rates_arrival_los = np.array(rates_arrival_los).reshape(n_class, 7, los)

In [8]:
rates_arrival_los

array([[[135, 270, 405],
        [135, 270, 405],
        [135, 270, 405],
        [135, 270, 378],
        [135, 243, 351],
        [108, 216, 351],
        [108, 243, 378]],

       [[115, 230, 345],
        [115, 230, 345],
        [115, 230, 345],
        [115, 230, 322],
        [115, 207, 299],
        [ 92, 184, 299],
        [ 92, 207, 322]]])

In [9]:
# Coefficients of objective function for LP
obj_coefs = (-1) * rates_arrival_los.reshape(n_class * 7 * los)
obj_coefs

array([-135, -270, -405, -135, -270, -405, -135, -270, -405, -135, -270,
       -378, -135, -243, -351, -108, -216, -351, -108, -243, -378, -115,
       -230, -345, -115, -230, -345, -115, -230, -345, -115, -230, -322,
       -115, -207, -299,  -92, -184, -299,  -92, -207, -322])

In [10]:
# Inequality equations, LHS
# We have total number of 42 decision veriables, corresponding to total number of
# rate class, arrival day of week and los combinations.
# Column indexes 0-20 are associated with decision variables for rate class 1
# Column indexes 21-41 are associated with decision variables for rate class 2
G = np.zeros(7 * los * n_class * 7).reshape(7, n_class*7*los)
# Arrivals that span Sunday stay night for rate class 1
G[0,:(7*los)] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Arrivals that span Monday stay night for rate class 1
G[1,:(7*los)] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[2,:(7*los)] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[3,:(7*los)] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[4,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
G[5,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
G[6,:(7*los)] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]
# Arrivals that span Sunday stay night for rate class 2
G[0,(7*los):] = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Arrivals that span Monday stay night for rate class 2
G[1,(7*los):] = [0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[2,(7*los):] = [0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[3,(7*los):] = [0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
G[4,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
G[5,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0]
G[6,(7*los):] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1]

# identity matrix for expected demand constraints, G for capacity constraints,
# Negative identity matrix for non-negativity
G = np.concatenate((np.identity(combs), G, -np.identity(combs)), axis=0)
# Inequality equations, RHS
# For each rate class, number of arrivals for a stay night in question is half of
# expected demand, which is capacity * intensity, then this expected demand is equally 
# split between 6 arrival day, los combination that spans the stay night in question
expDemand_each = (capacity * intensity * 0.5) / 6
h = np.round(expDemand_each, decimals=0) * np.ones(n_class * 7 * los)
# First h for expected demand, second component for capacity rhs
# Third component for non-negativity rhs.
h = np.concatenate((h, capacity * np.ones(7), np.zeros(combs)), axis=0)

In [11]:
h

array([ 6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6., 50., 50., 50., 50., 50., 50., 50.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [12]:
# Convert numpy arrays to cvxopt matrix forms
c = matrix(obj_coefs, tc='d')
G = matrix(G)
h = matrix(h)

In [13]:
# Solve LP, method = "interior point method" by default
# Results are used for initialization purpose (warm start), so it should
# not affect final algorithm performance after an efficient number of runs
sol = solvers.lp(c, G, h)

     pcost       dcost       gap    pres   dres   k/t
 0: -4.3576e+04 -1.0770e+05  5e+04  1e-01  8e-01  1e+00
 1: -4.4750e+04 -7.3030e+04  2e+04  7e-02  4e-01  5e+01
 2: -4.5974e+04 -5.1996e+04  3e+03  1e-02  8e-02  2e+01
 3: -4.6445e+04 -4.7593e+04  6e+02  3e-03  1e-02  3e+00
 4: -4.6520e+04 -4.6801e+04  2e+02  7e-04  4e-03  1e+00
 5: -4.6547e+04 -4.6554e+04  3e+00  1e-05  8e-05  3e-02
 6: -4.6548e+04 -4.6548e+04  3e-02  1e-07  8e-07  3e-04
 7: -4.6548e+04 -4.6548e+04  3e-04  1e-09  8e-09  3e-06
Optimal solution found.


In [14]:
# Optimal solutions serve as booking limits: number of booking requests to accept for 
# a given rate class, arrival day, los combination
bkLimits = np.array(sol['x']).reshape(n_class, 7, los)
bkLimits = np.round(bkLimits, decimals=0)
bkLimits

array([[[6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.],
        [6., 6., 6.],
        [2., 6., 6.]],

       [[6., 6., 6.],
        [4., 2., 2.],
        [1., 1., 2.],
        [3., 3., 3.],
        [3., 3., 0.],
        [2., 0., 6.],
        [0., 6., 6.]]])

In [15]:
# Dual values associated with demand constraints
# Represent marginal contribution for the stay night revenue
duals = np.array(sol['z'])[:(n_class*7*los)].reshape(n_class, 7, los)
duals = np.round(duals, decimals=0)
duals

array([[[135., 155., 175.],
        [ 20.,  40.,  60.],
        [ 20.,  40.,  60.],
        [ 20.,  40.,  56.],
        [ 20.,  36.,  36.],
        [ 16.,  16., 151.],
        [  0., 135., 270.]],

       [[115., 115., 115.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,   0.],
        [  0.,   0.,  99.],
        [  0.,  99., 214.]]])

In [16]:
# '012' represents rate class 1, Monday arrival and 3-night stay
sun_stay_index = ['000', '001', '002', '061', '062', '052', 
            '100', '101', '102', '161', '162', '152']

mon_stay_index = ['010', '011', '012', '001', '002', '062', 
            '110', '111', '112', '101', '102', '162']

tue_stay_index = ['020', '021', '022', '011', '012', '002', 
            '120', '121', '122', '111', '112', '102']

wed_stay_index = ['030', '031', '032', '021', '022', '012', 
            '130', '131', '132', '121', '122', '112']

thr_stay_index = ['040', '041', '042', '031', '032', '022', 
            '140', '141', '142', '131', '132', '122']

fri_stay_index = ['050', '051', '052', '041', '042', '032', 
            '150', '151', '152', '141', '142', '132']

sat_stay_index = ['060', '061', '062', '051', '052', '042', 
            '160', '161', '162', '151', '152', '142']

In [17]:
sun_duals = [duals[int(item[0]), int(item[1]), int(item[2])] for item in sun_stay_index]
sun_duals_max = np.max(sun_duals)
max_buckets = 5
sun_duals_interval = sun_duals_max / max_buckets
bucket = []
n_buckets = 1
for item in sun_stay_index:
    # rate class
    r_index = int(item[0])
    # arrival day of week
    a_index = int(item[1])
    # duration (los)
    d_index = int(item[2])
    